In [1]:
import pandas as pd
import numpy as np

# EDA
## Data Import

In [3]:
weather = pd.read_csv('data/SummaryOfWeather.csv', low_memory=False)

In [8]:
## Get the number of columns and rows
weather.shape

(119040, 31)

## Decisions
### Null Data

There are a total of 119,040 observations. 
- **STA** - Weather Station
- **Date** - Self-explanatory
- **Precip** - Precipitation in mm
- **WindGustSpd** - Peak wind gust speed in km/h 
    - **(118,508)**
    - Keep windgust speeds for now but leave out of model
    - Set null values to 0 for now. 
- **MaxTemp** - Maximum temperature in degrees Celsius
- **MinTemp** - Minimum temperature in degrees Celsius
- **MeanTemp** - Mean temperature in degrees Celsius
- **Snowfall** - Snowfall and Ice Pellets in mm 
    - **(1163)**
    - Keep snowfall
- **PoorWeather** -A repeat of the TSHDSBRSGF column 
    - **(84803)**
    - Keep PoorWeather and add ass a factor
    - 1 = Yes, 0 = Unknown
    - Determine if we can find this data or derive it based on temperature and precipitation
- **YR** - Year of Observation
- **MO** - Month of Observation
- **DA** - Day of Observation
- **PRCP** - Precipitation in Inches and Hundredths 
    - **(1932)**
    - Keep and use to determine weather conditions
-  **DR** - Peak wind gust direction in tens of degrees 
    - **(118507)**
    - Set null values to 0 for now
- **SPD** - Peak wind gust speed in knots 
    - **(118508)**
    - Set null values to 0 for now
- **MAX** - Maximum temperature in degrees Fahrenheit 
    - **(474)**
    - See what days are missing and see if they correlate to MIN, MAX. Also determine if important missions happended during this time. 
- **MIN** - Minimum temperature in degrees Fahrenheit 
    - **(468)** 
    - Same as max
- **MEA** - Mean temperature in degrees Fahrenheit 
    - **(498)** 
    - Same as Max
- **SNF** - Snowfall in inches and tenths 
    - **(1163)**
    - Set unknown to 0 for now
- **SND** - Snow depth (includes ice pellets) recorded at 1200 GMT except 0000 GMT in Far East Asian Area in inches and tenths 
    - **(113477)**
    - Set nulls to zero
- **FTFrozen** - Ground Top (depth in inches) 
    - **(119040)**
    - Delete for now unless we can find the data
- **FBFrozen** - Ground Base (depth in inches) 
    - **(119040)**
    - Delete for now unless we can find the data
- **FTIFrozen** - Ground Thickness (thickness in inches) 
    - **(119040)**
    - Delete for now unless we can find the data
- **ITHIce** - Thickness on Water (inches and tenths) 
    - **(119040)**
    - Delete for now unless we can find the data
- **PGTPeak** wind gust time (hours and tenths) 
    - **(118515)**
    - Set null values to 0 for now
- **TSHDSBRSGFDay** with: Thunder; Sleet; Hail; Dust or Sand; Smoke or Haze; Blowing Snow; Rain; Snow; Glaze; Fog; 0 = No, 1 = Yes 
    - **(84803)**
- **SD3Snow** depth at 0030 GMT includes ice pellets in inches and tenths
    - **(119040)**
    - Delete for now unless we can find the data
- **RHX24-hour** maximum relative humidity, as a whole percent 
    - **(119040)**
    - Delete for now unless we can find the data
- **RHN24-hour** minimum relative humidity, as a whole percent 
    - **(119040)**
    - Delete for now unless we can find the data
- **RVGRiver** guage in feet and tenths 
    - **(119040)**
    - Delete for now unless we can find the data
- **WTEWater** equivalent of snow and ice on ground in inches and hundredths 
    - **(119040)**
    - Delete for now unless we can find the data
    

In [6]:
### How many null columns
weather.isnull().sum()

STA                 0
Date                0
Precip              0
WindGustSpd    118508
MaxTemp             0
MinTemp             0
MeanTemp            0
Snowfall         1163
PoorWeather     84803
YR                  0
MO                  0
DA                  0
PRCP             1932
DR             118507
SPD            118508
MAX               474
MIN               468
MEA               498
SNF              1163
SND            113477
FT             119040
FB             119040
FTI            119040
ITH            119040
PGT            118515
TSHDSBRSGF      84803
SD3            119040
RHX            119040
RHN            119040
RVG            119040
WTE            119040
dtype: int64